In [51]:
import pandas as pd
import string
import re
from unidecode import unidecode
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from nltk.stem.snowball import FrenchStemmer
from gensim.models import Phrases
import joblib


In [52]:
df_offres = pd.read_csv('df_offresetsalaires.csv')

In [53]:
df_offres = df_offres.drop(columns=df_offres.columns[0]).drop_duplicates(subset='id')

In [54]:
def cleaning(text):
#Retirer les urls et les retours lignes
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub(r'\n', '', text)
#Retirer les chiffres
    text= "".join([i for i in text if not i.isdigit()])
#Retirer la ponctuation et les accents
    table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(table)
    text = unidecode(text)

    return text.lower().split()

In [55]:
df_offres['description_cleaned'] = df_offres['description'].apply(lambda row: cleaning(row))

In [56]:
with open("stop_words_french.txt", "r", encoding="utf-8") as file:
    stopwords = [ligne.strip() for ligne in file]

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [57]:
df_offres["description_cleanednosw"] = df_offres["description_cleaned"].apply(lambda x: remove_stopwords(x))

In [58]:
stemmer = FrenchStemmer()

# Créer la fonction de stemming
def stemming (listed):
  output = [stemmer.stem(i) for i in listed]
  return output

import ast
def lister1(text):
    result = ast.literal_eval(text)
    return result

df_offres["description_stemmed"] = df_offres["description_cleanednosw"].apply(lambda x: stemming(x))

In [59]:
ensemble_listes_mots = list(df_offres['description_stemmed'])

# Initialisation d'un dictionnaire pour stocker les occurrences de chaque mot
occurrences_mots = Counter()

# Parcours de chaque liste de mots et mise à jour des occurrences
for liste_mots in ensemble_listes_mots:
    for mot in liste_mots:
        occurrences_mots[mot] = occurrences_mots.get(mot, 0) + 1

# Affichage des occurrences de chaque mot
occurrences_mots

Counter({'ete': 5133,
         'servic': 5052,
         'suiv': 4826,
         'particip': 4688,
         'techniqu': 4663,
         'charg': 4109,
         'accompagn': 3913,
         'activit': 3660,
         'mis': 3401,
         'avez': 3358,
         'sein': 3311,
         'travaill': 3310,
         'jour': 3282,
         'client': 3149,
         'realis': 3051,
         'fonction': 2958,
         'accueil': 2907,
         'securit': 2833,
         'mainten': 2815,
         'produit': 2772,
         'etabl': 2711,
         'cadr': 2695,
         'respons': 2629,
         'control': 2567,
         'social': 2534,
         'commercial': 2501,
         'an': 2499,
         'ser': 2492,
         'bon': 2458,
         'capacit': 2428,
         'entretien': 2394,
         'maitris': 2372,
         'sen': 2364,
         'prepar': 2306,
         'plac': 2274,
         'assur': 2225,
         'projet': 2221,
         'fair': 2211,
         'prim': 2202,
         'product': 2189,
         '

In [60]:
# Occurence minimal du terme dans le texte
min_count = 2

# Seuil d'acceptation des bigrams
threshold = 10

#Création du modèle de bigram
bigram_mod = Phrases(df_offres["description_stemmed"], min_count, threshold)

#Création de la fonction de bigram
def bigram(texts):
  return [bigram_mod[doc] for doc in texts]

df_offres["description_bigram"] = bigram(df_offres["description_stemmed"])

In [75]:
df_offres["description_stemmed"][0]

['cahor',
 'restaur',
 'aromat',
 'nouveau',
 'chef',
 'cuisin',
 'ete',
 'chef',
 'sein',
 'cuisin',
 'travaill',
 'produit',
 'saison',
 'elabore',
 'part',
 'produit',
 'march',
 'fait',
 'sincerit',
 'passion',
 'simplicit',
 'avez',
 'signif',
 'restaur',
 'gastronom',
 'sem',
 'gastronom',
 'offre',
 'sem',
 'bistronom',
 'propose',
 'week',
 'end',
 'entre',
 'plat',
 'dessert',
 'arcambal',
 'ouvert',
 'mid',
 'lund',
 'diner',
 'vendred',
 'samed',
 'juillet',
 'aout',
 'ouvertur',
 'din',
 'mercred',
 'samed',
 'eur',
 'net',
 'cdi',
 'avantag',
 'mutuel',
 'entreprisepost',
 'pourvoir',
 'possiblevous',
 'avez',
 'imper',
 'restaur',
 'gastronom',
 'precis',
 'cv']

In [61]:
df_offres.to_csv('df_offresv2.csv')

In [62]:
frozen_model = bigram_mod.freeze()
frozen_model.save("bigram_mod.pkl")


In [63]:
ensemble_listes_mots = list(df_offres['description_bigram'])

# Initialisation d'un dictionnaire pour stocker les occurrences de chaque mot
occurrences_mots = Counter()

# Parcours de chaque liste de mots et mise à jour des occurrences
for liste_mots in ensemble_listes_mots:
    for mot in liste_mots:
        occurrences_mots[mot] = occurrences_mots.get(mot, 0) + 1

# Affichage des occurrences de chaque mot
occurrences_mots

Counter({'suiv': 4789,
         'servic': 4298,
         'techniqu': 4212,
         'particip': 3664,
         'accompagn': 3402,
         'activit': 3170,
         'fonction': 2652,
         'client': 2618,
         'realis': 2580,
         'sein': 2563,
         'ete': 2484,
         'accueil': 2392,
         'etabl': 2317,
         'travaill': 2259,
         'produit': 2251,
         'avez': 2153,
         'control': 2146,
         'assur': 2138,
         'cadr': 2025,
         'respons': 1996,
         'commercial': 1977,
         'mainten': 1936,
         'product': 1932,
         'prepar': 1826,
         'entretien': 1804,
         'securit': 1792,
         'principal': 1722,
         'anim': 1704,
         'social': 1651,
         'effectu': 1644,
         'charg': 1598,
         'ger': 1568,
         'integr': 1562,
         'an': 1535,
         'projet': 1514,
         'vent': 1506,
         'group': 1479,
         'materiel': 1454,
         'aid': 1385,
         'quotidien': 

### Création du modèle

In [64]:
df_model = df_offres[['salaires_final','description_stemmed','description_bigram']]
df_model['description_stemmed'] = df_model['description_stemmed'].apply(lambda x: " ".join(x))
df_model['description_bigram'] = df_model['description_bigram'].apply(lambda x: " ".join(x))
df_model[(df_model['salaires_final'] > 1200) & (df_model['salaires_final'] < 10000)]

df_model_train,df_model_test = train_test_split(df_model, test_size=0.3)
df_model_train = df_model_train.reset_index(drop=True)
df_model_test = df_model_test.reset_index(drop=True)

C:\Users\timot\AppData\Local\Temp\ipykernel_7176\1713482175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['description_stemmed'] = df_model['description_stemmed'].apply(lambda x: " ".join(x))
C:\Users\timot\AppData\Local\Temp\ipykernel_7176\1713482175.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['description_bigram'] = df_model['description_bigram'].apply(lambda x: " ".join(x))


In [65]:
df_model.to_csv('df_model.csv')

In [66]:
# Variable du nombre de voisins
k = 15

vectorizer = TfidfVectorizer()
# Vectorisation des descriptions
X_bigram = vectorizer.fit_transform(df_model_train['description_bigram'])
# Calcul des similarités avec la nouvelle description
def prediction_bigram(description):
    new_vector = vectorizer.transform([description])
    similarities = cosine_similarity(X_bigram, new_vector)

    df_temp = df_model_train.copy()
    df_temp['similarities'] = similarities
    df_temp = df_temp.sort_values('similarities',ascending=False).head(k)

    return (df_temp['salaires_final'] * df_temp['similarities']).sum()/df_temp['similarities'].sum()

In [67]:
joblib.dump(vectorizer, 'vectorizer_bigram.pkl')

['vectorizer_bigram.pkl']

In [68]:
df_model_test['prediction_bigram'] = df_model_test['description_bigram'].apply(lambda x: prediction_bigram(x))

In [69]:
X_stemmed = vectorizer.fit_transform(df_model_train['description_stemmed'])
# Calcul des similarités avec la nouvelle description
def prediction_stemmed(description):
    new_vector = vectorizer.transform([description])
    similarities = cosine_similarity(X_stemmed, new_vector)

    df_temp = df_model_train.copy()
    df_temp['similarities'] = similarities
    df_temp = df_temp.sort_values('similarities',ascending=False).head(k)

    return (df_temp['salaires_final'] * df_temp['similarities']).sum()/df_temp['similarities'].sum()

In [70]:
df_model_test['prediction_stemmed'] = df_model_test['description_stemmed'].apply(lambda x: prediction_stemmed(x))

In [71]:
r2_bigram = r2_score(df_model_test['salaires_final'], df_model_test['prediction_bigram'])
r2_stemmed = r2_score(df_model_test['salaires_final'], df_model_test['prediction_stemmed'])

In [72]:
print(f'r2_bigram : {r2_bigram}')
print(f'r2_stemmed : {r2_stemmed}')

r2_bigram : -0.0010968970795177668
r2_stemmed : -0.010143709422800962
